# WM Segmentation using U-Net
In this project, I will use MRI T1-weighted images to segmentation white matter (WM) from other tissues in the brain, such as grey matter and cerebrospinal fluid. There are many preprocessing toolboxes that perform GM/WM segmentation, including [FSL](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki), [SPM](https://www.fil.ion.ucl.ac.uk/spm/) and [BrainSuite](http://brainsuite.org/). This problem has also frequently been tackled using deep learning, with [U-Net](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/), which is the convolutional neural network for biomedical segmentation.

In this script, I will implement this U-Net using tensorflow and calculate the Dice Similarity Coefficient on the test set white matter segmentations. Ground truth white matter masks are generated using BrainSuite, with voxels of > 50% probability of being within white matter. Instead of training the network using the full image, I will use 64x64 patches extracted from the MRI dataset. No data augmentation is performed to simplify this process.

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_extraction import image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Conv2DTranspose, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
np.random.seed(1)
patch_size = 64
already_generated = True

root_dir = 'Brain_T1.nosync/'

The training and testing image patches have already been extracted, split and saved in the home folder. 

In [3]:
with open(root_dir+'t1_patches_train2.npy', 'rb') as f:
	t1_patches_train = np.load(f)
with open(root_dir+'t1_patches_test2.npy', 'rb') as f:
	t1_patches_test = np.load(f)
with open(root_dir+'wm_patches_train2.npy', 'rb') as f:
	wm_patches_train = np.load(f)
with open(root_dir+'wm_patches_test2.npy', 'rb') as f:
	wm_patches_test = np.load(f)

In [4]:
len_train = len(t1_patches_train)
len_test = len(t1_patches_test)

### U-Net
I will implement a U-Net architecture, in which regional and global features are trained using the 2D convolution layer `Conv2D` and aggregated to generate the white matter probability map. I will specify the learning rate as 0.001.

In [5]:
def unet_model(input_shape=(64,64,1), dropout_rate=0.20, kernel_size=(3,3), optimizer='adam',
				dense_params=64, maxpool_size=(2,2), activation='relu'):
	input_image = Input(input_shape)
	c1 = Conv2D(16, kernel_size=kernel_size, activation=activation, padding='same')(input_image)	
	c1 = Conv2D(16, kernel_size=kernel_size, activation=activation, padding='same')(c1)	
	p1 = MaxPooling2D(pool_size=maxpool_size)(c1)
	p1 = Dropout(dropout_rate)(p1)

	c2 = Conv2D(32, kernel_size=kernel_size, activation=activation, padding='same')(p1)	
	c2 = Conv2D(32, kernel_size=kernel_size, activation=activation, padding='same')(c2)	
	p2 = MaxPooling2D(pool_size=maxpool_size)(c2)
	p2 = Dropout(dropout_rate)(p2)

	c3 = Conv2D(64, kernel_size=kernel_size, activation=activation, padding='same')(p2)
	c3 = Conv2D(64, kernel_size=kernel_size, activation=activation, padding='same')(c3)

	u4 = Conv2DTranspose(32, kernel_size=(2,2), strides=maxpool_size)(c3)
	u4 = Concatenate(axis=3)([u4, c2])
	u4 = Dropout(dropout_rate)(u4)
	c4 = Conv2D(32, kernel_size=kernel_size, activation=activation, padding='same')(u4)
	c4 = Conv2D(32, kernel_size=kernel_size, activation=activation, padding='same')(c4)

	u5 = Conv2DTranspose(16, kernel_size=(2,2), strides=maxpool_size)(c4)
	u4 = Concatenate(axis=3)([u5, c1])
	u5 = Dropout(dropout_rate)(u5)
	c5 = Conv2D(16, kernel_size=kernel_size, activation=activation, padding='same')(u5)
	c5 = Conv2D(16, kernel_size=kernel_size, activation=activation, padding='same')(c5)

	output_image = Conv2D(1, kernel_size=(1,1), activation='sigmoid')(c5)
	model = Model(inputs=input_image, outputs=output_image)
	model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [6]:
unet = unet_model(optimizer=Adam(learning_rate=0.001))

Here, I will train the U-Net for 10 epochs. I have trained this network offline and saved the checkpoint for convenience.

In [7]:
if not already_generated:
    filepath="checkpoints.nosync/unet-adam001-{epoch:02d}-{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    history = unet.fit(x=t1_patches_train, y=wm_patches_train, epochs=10, batch_size=128, callbacks=callbacks_list)
else:
    unet.load_weights('checkpoints.nosync/unet-adam001-10-0.0897.hdf5')

### White Matter Masks
I will generate the white matter masks on the T1 MRI patches of the test set. Since the output of the U-Net is the probably map (after sigmoid activation function) of whether a voxel belongs in the white matter or not, I will consider voxels > 50% white matter to be within the mask. (This is the same 50% threshold as when the ground truth masks are generated from BrainSuite).

In [8]:
preds = unet.predict(t1_patches_test)
preds_reshape = preds.squeeze() > 0.5

Due to HIPPA, I am unable to show the results of the segmentation masks side-by-side with the ground truth masks. However, from our predictions, we can calculate the Dice Similarity Coefficient (DSC), which is the score in the [0,1] range of how well the segmented masks matched the ground truth. The final DSC is 0.94, which is a very high result for segmentation, which proves that the white matter segmentation task can be learned very efficiently (after only 10 epochs) using a simple U-Net.

In [9]:
dsc = np.empty(len_test)
for index in range(len_test):
	label = wm_patches_test[index,:,:]
	pred = preds_reshape[index,:,:]
	tp = np.sum(label * pred)
	fp = np.sum((label==0) * pred)
	fn = np.sum(label * (pred==0))
	dsc[index] = (2*tp) / (2*tp + fp + fn)
print(np.mean(dsc))

0.942418467133304


# Conclusion
In this work, we use a U-Net neural network to perform white matter segmentation on T1-weighted MRI images. The resulting Dice Similarity Coefficient is 0.94 on the test set, which is a large improvement compared to the use of random forest classifier on each voxel (0.75 Dice Similarity, [link](https://github.com/chauvu/chauvu.github.io/blob/main/Notebooks/wm_segmentation_randomforest.ipynb)). 